In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
#load the datset
amazon=pd.read_csv("amazon.csv")

In [3]:
columns_to_keep = ['review_content', 'review_title', 'rating', 'product_id', 'category']
amazon_new = amazon[columns_to_keep]

In [4]:
amazon_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review_content  1465 non-null   object
 1   review_title    1465 non-null   object
 2   rating          1465 non-null   object
 3   product_id      1465 non-null   object
 4   category        1465 non-null   object
dtypes: object(5)
memory usage: 57.4+ KB


In [5]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

In [6]:
import re

In [7]:
def clean_text(text):
    text = str(text).lower()                        # convert to lowercase
    text = re.sub(r"http\S+|www\S+", "", text)      # remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)         # remove punctuation, numbers, emojis
    text = re.sub(r"\s+", " ", text).strip()        # fix multiple spaces
    return text

In [8]:
amazon_new['clean_review'] = amazon_new['review_content'].apply(clean_text)

C:\Users\himan\AppData\Local\Temp\ipykernel_46580\316615009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_new['clean_review'] = amazon_new['review_content'].apply(clean_text)


In [9]:
!pip install textblob


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Users\himan\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [10]:
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\himan\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\himan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\himan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\himan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
from textblob import TextBlob

def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

amazon_new['sentiment'] = amazon_new['clean_review'].apply(get_sentiment)

C:\Users\himan\AppData\Local\Temp\ipykernel_46580\1454819888.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_new['sentiment'] = amazon_new['clean_review'].apply(get_sentiment)


In [12]:
amazon_new['sentiment'].value_counts()

sentiment
Positive    1362
Neutral       97
Negative       6
Name: count, dtype: int64

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  confusion_matrix, classification_report
from sklearn.svm import SVC

In [14]:
# Sample 100 reviews
sample_df = amazon_new.sample(n=100, random_state=42)

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(sample_df['clean_review'])

# Encode sentiment labels
encoder = LabelEncoder()
y = encoder.fit_transform(sample_df['sentiment'])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Train Logistic Regression
log_model = LogisticRegression(class_weight='balanced')
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_test)

# Train SVM
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)


In [15]:
# Compare performance
print("Logistic Regression Performance:")
print(classification_report(y_test, log_preds, target_names=encoder.classes_))

print("SVM Performance:")
print(classification_report(y_test, svm_preds, target_names=encoder.classes_))


Logistic Regression Performance:
              precision    recall  f1-score   support

     Neutral       0.00      0.00      0.00         2
    Positive       0.93      1.00      0.97        28

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

SVM Performance:
              precision    recall  f1-score   support

     Neutral       0.00      0.00      0.00         2
    Positive       0.93      1.00      0.97        28

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30



C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [16]:
#data is  highly imbalanced, so the results are also showing that now manually fixing this

In [17]:
# Manually balance the dataset
min_count = amazon_new['sentiment'].value_counts().min()
sample_size = min(min_count, 50) 


pos = amazon_new[amazon_new['sentiment'] == 'Positive'].sample(n=sample_size, random_state=1)
neg = amazon_new[amazon_new['sentiment'] == 'Negative'].sample(n=sample_size, random_state=2)
neu = amazon_new[amazon_new['sentiment'] == 'Neutral'].sample(n=sample_size, random_state=3)
balanced_df = pd.concat([pos, neg, neu])

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(balanced_df['clean_review'])

# Encode sentiment labels
encoder = LabelEncoder()
y = encoder.fit_transform(balanced_df['sentiment'])

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Train Logistic Regression
log_model = LogisticRegression(class_weight='balanced')
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_test)

# Train SVM
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)

# Compare performance
print("Logistic Regression Performance:")
print(classification_report(y_test, log_preds, target_names=encoder.classes_))

print("SVM Performance:")
print(classification_report(y_test, svm_preds, target_names=encoder.classes_))


Logistic Regression Performance:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00         2
     Neutral       1.00      0.50      0.67         2
    Positive       0.67      1.00      0.80         2

    accuracy                           0.83         6
   macro avg       0.89      0.83      0.82         6
weighted avg       0.89      0.83      0.82         6

SVM Performance:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00         2
     Neutral       1.00      0.50      0.67         2
    Positive       0.67      1.00      0.80         2

    accuracy                           0.83         6
   macro avg       0.89      0.83      0.82         6
weighted avg       0.89      0.83      0.82         6



In [18]:
#still didnt help gonna try different model

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
# Balance the dataset based on available samples
min_count = amazon_new['sentiment'].value_counts().min()
pos = amazon_new[amazon_new['sentiment'] == 'Positive'].sample(n=min_count, random_state=1)
neg = amazon_new[amazon_new['sentiment'] == 'Negative'].sample(n=min_count, random_state=2)
neu = amazon_new[amazon_new['sentiment'] == 'Neutral'].sample(n=min_count, random_state=3)
balanced_df = pd.concat([pos, neg, neu])

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(balanced_df['clean_review'])

# Encode sentiment labels
encoder = LabelEncoder()
y = encoder.fit_transform(balanced_df['sentiment'])

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

# Evaluate performance
print("Random Forest Performance:")
print(classification_report(y_test, rf_preds, target_names=encoder.classes_))


Random Forest Performance:
              precision    recall  f1-score   support

    Negative       0.33      1.00      0.50         2
     Neutral       0.00      0.00      0.00         2
    Positive       0.00      0.00      0.00         2

    accuracy                           0.33         6
   macro avg       0.11      0.33      0.17         6
weighted avg       0.11      0.33      0.17         6



C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [21]:
import pandas as pd
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier



# Balance the dataset based on available samples
min_count = amazon_new['sentiment'].value_counts().min()
pos = amazon_new[amazon_new['sentiment'] == 'Positive'].sample(n=min_count, random_state=1)
neg = amazon_new[amazon_new['sentiment'] == 'Negative'].sample(n=min_count, random_state=2)
neu = amazon_new[amazon_new['sentiment'] == 'Neutral'].sample(n=min_count, random_state=3)
balanced_df = pd.concat([pos, neg, neu])

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(balanced_df['clean_review'])

# Encode sentiment labels
encoder = LabelEncoder()
y = encoder.fit_transform(balanced_df['sentiment'])

# Train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Train XGBoost Classifier
xgb_model = XGBClassifier(XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    scale_pos_weight=1,
    random_state=42
))
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# Evaluate performance
print("XGBoost Performance:")
print(classification_report(y_test, xgb_preds, target_names=encoder.classes_))

C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:723: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:15:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "objective__colsample_bytree", "objective__enable_categorical", "objective__eval_metric", "objective__learning_rate", "objective__max_depth", "objective__missing", "objective__n_estimators", "objective__objective", "objective__random_state", "objective__scale_pos_weight", "objective__subsample", "objective__use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Performance:
              precision    recall  f1-score   support

    Negative       0.50      1.00      0.67         2
     Neutral       1.00      0.50      0.67         2
    Positive       1.00      0.50      0.67         2

    accuracy                           0.67         6
   macro avg       0.83      0.67      0.67         6
weighted avg       0.83      0.67      0.67         6



In [22]:
# Step 3: Convert to binary sentiment
amazon_new['binary_sentiment'] = amazon_new['sentiment'].apply(
    lambda x: 'Positive' if x == 'Positive' else 'Not Positive'
)

# Step 4: TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(amazon_new['clean_review'])

# Step 5: Encode binary labels
encoder = LabelEncoder()
y = encoder.fit_transform(amazon_new['binary_sentiment'])  # 0 = Not Positive, 1 = Positive

# Step 6: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Step 7: Train XGBoost classifier
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=1,
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# Step 8: Evaluate performance
print("Binary Sentiment Classification (XGBoost):")
print(classification_report(y_test, xgb_preds, target_names=encoder.classes_))


C:\Users\himan\AppData\Local\Temp\ipykernel_46580\4269241452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_new['binary_sentiment'] = amazon_new['sentiment'].apply(
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:15:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Binary Sentiment Classification (XGBoost):
              precision    recall  f1-score   support

Not Positive       1.00      0.13      0.23        31
    Positive       0.94      1.00      0.97       409

    accuracy                           0.94       440
   macro avg       0.97      0.56      0.60       440
weighted avg       0.94      0.94      0.92       440



In [23]:
#still not working . so gonna try Undersampled Binary Sentiment Classification with XGBoost
 # Step 3: Convert to binary sentiment
amazon_new['binary_sentiment'] = amazon_new['sentiment'].apply(
    lambda x: 'Positive' if x == 'Positive' else 'Not Positive'
)

# Step 4: Undersample Positive class to match Not Positive
not_positive = amazon_new[amazon_new['binary_sentiment'] == 'Not Positive']
positive = amazon_new[amazon_new['binary_sentiment'] == 'Positive'].sample(n=len(not_positive), random_state=1)
balanced_df = pd.concat([positive, not_positive])

# Step 5: TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(balanced_df['clean_review'])
# Step 6: Encode binary labels
encoder = LabelEncoder()
y = encoder.fit_transform(balanced_df['binary_sentiment'])  # 0 = Not Positive, 1 = Positive

# Step 7: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Step 8: Train XGBoost classifier
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=1,
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# Step 9: Evaluate performance
print("Binary Sentiment Classification (XGBoost, Undersampled):")
print(classification_report(y_test, xgb_preds, target_names=encoder.classes_))



C:\Users\himan\AppData\Local\Temp\ipykernel_46580\2123623656.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_new['binary_sentiment'] = amazon_new['sentiment'].apply(
C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:15:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Binary Sentiment Classification (XGBoost, Undersampled):
              precision    recall  f1-score   support

Not Positive       0.73      0.77      0.75        31
    Positive       0.76      0.71      0.73        31

    accuracy                           0.74        62
   macro avg       0.74      0.74      0.74        62
weighted avg       0.74      0.74      0.74        62



In [24]:
# Count total samples per class
class_counts = amazon_new['binary_sentiment'].value_counts()
print("Total samples per class:")
print(class_counts)

# Check how many you're sampling
sample_size = 31 
print("\nSampling size:", sample_size)

# Compare with actual counts
for sentiment, count in class_counts.items():
    if sample_size >= count:
        print(f"⚠️ You're using ALL {sentiment} samples ({count}) — not a true sample.")
    else:
        print(f"✅ You're sampling {sample_size} out of {count} {sentiment} reviews.")

Total samples per class:
binary_sentiment
Positive        1362
Not Positive     103
Name: count, dtype: int64

Sampling size: 31
✅ You're sampling 31 out of 1362 Positive reviews.
✅ You're sampling 31 out of 103 Not Positive reviews.


In [25]:

from sklearn.naive_bayes import MultinomialNB

# Step 4: Undersample Positive class to match Not Positive
not_positive = amazon_new[amazon_new['binary_sentiment'] == 'Not Positive']
positive = amazon_new[amazon_new['binary_sentiment'] == 'Positive'].sample(n=len(not_positive), random_state=1)
balanced_df = pd.concat([positive, not_positive])

# Step 5: TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(balanced_df['clean_review'])

# Step 6: Encode binary labels
encoder = LabelEncoder()
y = encoder.fit_transform(balanced_df['binary_sentiment'])  # 0 = Not Positive, 1 = Positive

# Step 7: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Step 8: Train Naive Bayes classifier
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_preds = nb_model.predict(X_test)

# Step 9: Evaluate performance
print("Binary Sentiment Classification (Naive Bayes):")
print(classification_report(y_test, nb_preds, target_names=encoder.classes_))

Binary Sentiment Classification (Naive Bayes):
              precision    recall  f1-score   support

Not Positive       0.83      0.61      0.70        31
    Positive       0.69      0.87      0.77        31

    accuracy                           0.74        62
   macro avg       0.76      0.74      0.74        62
weighted avg       0.76      0.74      0.74        62



In [26]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Step 1: Define individual models
nb_model = MultinomialNB()
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Step 2: Create ensemble with hard voting
ensemble_model = VotingClassifier(
    estimators=[
        ('nb', nb_model),
        ('xgb', xgb_model)
    ],
    voting='soft'  # Use 'soft' if you want to average probabilities
)

# Step 3: Train/test split (reuse your TF-IDF features and labels)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Step 4: Train ensemble
ensemble_model.fit(X_train, y_train)

# Step 5: Predict and evaluate
ensemble_preds = ensemble_model.predict(X_test)
print("Binary Sentiment Classification (Ensemble: Naive Bayes + XGBoost):")
print(classification_report(y_test, ensemble_preds, target_names=encoder.classes_))

Binary Sentiment Classification (Ensemble: Naive Bayes + XGBoost):
              precision    recall  f1-score   support

Not Positive       0.73      0.77      0.75        31
    Positive       0.76      0.71      0.73        31

    accuracy                           0.74        62
   macro avg       0.74      0.74      0.74        62
weighted avg       0.74      0.74      0.74        62



C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:15:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [27]:


# Step 8: Define models
nb_model = MultinomialNB()
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Step 9: Create soft voting ensemble
ensemble_model = VotingClassifier(
    estimators=[('nb', nb_model), ('xgb', xgb_model)],
    voting='soft'
)

# Step 10: Train ensemble
ensemble_model.fit(X_train, y_train)

# Step 11: Evaluate on test set
ensemble_preds = ensemble_model.predict(X_test)
print("Binary Sentiment Classification (Ensemble: Naive Bayes + XGBoost):")
print(classification_report(y_test, ensemble_preds, target_names=encoder.classes_))

# Step 12: Deploy on full dataset
X_full = vectorizer.transform(amazon_new['clean_review'])
full_preds = ensemble_model.predict(X_full)
amazon_new['predicted_sentiment'] = encoder.inverse_transform(full_preds)

# Step 13: View results
print(amazon_new[['review_content', 'predicted_sentiment']].head())
print(amazon_new['predicted_sentiment'].value_counts())

C:\Users\himan\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:15:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Binary Sentiment Classification (Ensemble: Naive Bayes + XGBoost):
              precision    recall  f1-score   support

Not Positive       0.73      0.77      0.75        31
    Positive       0.76      0.71      0.73        31

    accuracy                           0.74        62
   macro avg       0.74      0.74      0.74        62
weighted avg       0.74      0.74      0.74        62

                                      review_content predicted_sentiment
0  Looks durable Charging is fine tooNo complains...            Positive
1  I ordered this cable to connect my phone to An...            Positive
2  Not quite durable and sturdy,https://m.media-a...            Positive
3  Good product,long wire,Charges good,Nice,I bou...            Positive
4  Bought this instead of original apple, does th...            Positive
predicted_sentiment
Positive        1047
Not Positive     418
Name: count, dtype: int64


C:\Users\himan\AppData\Local\Temp\ipykernel_46580\2409056044.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amazon_new['predicted_sentiment'] = encoder.inverse_transform(full_preds)


In [28]:
import joblib

# Save the trained ensemble model
joblib.dump(ensemble_model, 'ensemble_model.joblib')

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'vectorizer.joblib')

# Save the label encoder
joblib.dump(encoder, 'label_encoder.joblib')

['label_encoder.joblib']